# Échantillonnage Stratifié Hybride pour Grands Ensembles de Données

Ce notebook démontre une approche d'**échantillonnage stratifié hybride** pour traiter de grands fichiers parquet. La technique combine une stratification **temporelle** et **spatiale** pour garantir des échantillons représentatifs tout en maintenant la qualité des données.

## 📊 Qu'est-ce que l'Échantillonnage Stratifié ?

L'échantillonnage stratifié divise la population en sous-groupes homogènes (strates) et échantillonne proportionnellement dans chacun. Cela garantit :
- **Meilleure représentation** des événements rares
- **Réduction du biais d'échantillonnage**
- **Amélioration de la validité statistique**

## 🎯 Notre Approche

Nous utilisons une **stratification à 3 niveaux** :
1. **Temporelle** : Heure de prise en charge (0-23)
2. **Spatiale** : Fréquence de localisation (zones rares/moyennes/fréquentes)
3. **Combinée** : Produit croisé temps × espace

---

## 📦 Étape 1 : Importer les Bibliothèques Requises

Nous avons besoin de :
- `pandas` : Manipulation de données
- `numpy` : Opérations numériques
- `psutil` : Surveillance des ressources système
- `threading` : Surveillance en arrière-plan
- `glob` : Correspondance de motifs de fichiers

In [1]:
import pandas as pd
import time
import psutil
import os
import threading
import glob
import numpy as np

## 📈 Étape 2 : Configuration du Monitoring en Temps Réel

Ce système de surveillance suit :
- **Utilisation CPU** (%)
- **Consommation RAM** (MB)
- **Progression** (%)
- **ETA** (temps restant estimé)

La surveillance s'exécute dans un **thread en arrière-plan** et se met à jour toutes les 2 secondes.

In [2]:
# Suivi global de la progression
progress = {
    'operation': 'Démarrage',
    'completed': 0.0,
    'start_time': time.time(),
    'eta': float('inf')
}
monitoring = True

def monitor_resources(interval=2):
    """Thread en arrière-plan qui surveille l'utilisation CPU et RAM"""
    process = psutil.Process(os.getpid())
    while monitoring:
        mem = process.memory_info().rss / 1024**2  # Convertir en MB
        cpu = psutil.cpu_percent()
        print(
            f"[{progress['operation']}] "
            f"{progress['completed']*100:.1f}% | "
            f"CPU: {cpu:.1f}% | "
            f"RAM: {mem:.1f} MB | "
            f"ETA: {progress['eta']:.1f}s"
        )
        time.sleep(interval)

# Démarrer la surveillance en arrière-plan
threading.Thread(target=monitor_resources, daemon=True).start()

[Démarrage] 0.0% | CPU: 54.0% | RAM: 106.1 MB | ETA: infs


## ⚙️ Étape 3 : Configuration des Paramètres

### 🔧 **PERSONNALISEZ CES PARAMÈTRES**

| Paramètre | Description | Défaut | Comment le Modifier |
|-----------|-------------|--------|---------------------|
| `SAMPLE_FRAC` | Fraction de données à échantillonner (0.0-1.0) | `0.01` (1%) | Augmenter pour des échantillons plus grands (ex: `0.05` = 5%) |
| `RANDOM_STATE` | Graine aléatoire pour la reproductibilité | `42` | Changer pour tout entier pour différents échantillons aléatoires |
| `FILE_PATTERN` | Motif glob pour les fichiers d'entrée | `"Data_as_paquets/yellow_tripdata_*.parquet"` | Modifier le chemin/motif pour correspondre à vos fichiers |
| `OUTPUT_FILE` | Nom du fichier CSV de sortie | `"yellowtaxisample1pct_hybrid_stratified.csv"` | Changer pour le nom de sortie souhaité |
| `ZONE_BINS` | Seuils pour la stratification spatiale | `[0, 0.01, 0.05, 1.0]` | Ajuster pour changer les définitions rare/moyen/fréquent |

### 📝 Exemples :
```python
# Pour un échantillon de 5% :
SAMPLE_FRAC = 0.05

# Et pour un échantillon de 1% :
SAMPLE_FRAC = 0.01

# Pour une stratification spatiale différente (plus granulaire) :
ZONE_BINS = [0, 0.005, 0.02, 0.1, 1.0]
ZONE_LABELS = ['tres_rare', 'rare', 'moyen', 'frequent']
```

In [3]:
# ========================================
# PARAMÈTRES - MODIFIEZ-LES SELON VOS BESOINS
# ========================================

SAMPLE_FRAC = 0.01          # Échantillonner 1% des données de chaque strate
RANDOM_STATE = 42           # Pour des résultats reproductibles
FILE_PATTERN = "Data_as_paquets/yellow_tripdata_*.parquet"
OUTPUT_FILE = "yellowtaxisample1pct_hybrid_stratified.csv"

# Bins de stratification spatiale
ZONE_BINS = [0, 0.01, 0.05, 1.0]
ZONE_LABELS = ['rare', 'moyen', 'frequent']

# ========================================

# Trouver tous les fichiers correspondants
files = sorted(glob.glob(FILE_PATTERN))
n_files = len(files)

print(f"Trouvé {n_files} fichiers à traiter")
print(f"Fraction d'échantillonnage : {SAMPLE_FRAC*100}%")
print(f"Fichier de sortie : {OUTPUT_FILE}")

Trouvé 36 fichiers à traiter
Fraction d'échantillonnage : 1.0%
Fichier de sortie : yellowtaxisample1pct_hybrid_stratified.csv


## 🔄 Étape 4 : Boucle d'Échantillonnage Stratifié Hybride

### Aperçu du Processus :

Pour chaque fichier parquet :

1. **Charger les données** dans un DataFrame pandas
2. **Extraire la caractéristique temporelle** : Heure de la datetime de prise en charge (0-23)
3. **Calculer les fréquences spatiales** : Fréquence d'apparition de chaque lieu de prise en charge
4. **Classifier les zones** en rare/moyen/fréquent selon la fréquence
5. **Créer des strates combinées** : ex. "14_rare" = heure 14 + zone rare
6. **Échantillonner proportionnellement** de chaque strate
7. **Nettoyer** les colonnes temporaires
8. **Mettre à jour la progression** et l'ETA

### Pourquoi Cela Fonctionne :

- **Les zones rares** (< 1% de fréquence) sont préservées dans l'échantillon
- **Les heures de pointe** maintiennent leurs motifs temporels
- **L'échantillonnage proportionnel** garantit la validité statistique

In [4]:
sample_parts = []
start_time = time.time()
progress['operation'] = 'Échantillonnage stratifié hybride'

for i, file in enumerate(files):
    # Charger le fichier parquet
    df = pd.read_parquet(file)
    
    # ========================================
    # 1️⃣ STRATIFICATION TEMPORELLE
    # ========================================
    # Extraire l'heure de la datetime de prise en charge (0-23)
    df['pickup_hour'] = pd.to_datetime(
        df['tpep_pickup_datetime']
    ).dt.hour
    
    # ========================================
    # 2️⃣ STRATIFICATION PAR FRÉQUENCE SPATIALE
    # ========================================
    # Calculer la fréquence normalisée de chaque lieu de prise en charge
    zone_freq = df['PULocationID'].value_counts(normalize=True)
    
    # Classifier les zones selon la fréquence
    df['zone_class'] = pd.cut(
        df['PULocationID'].map(zone_freq),
        bins=ZONE_BINS,
        labels=ZONE_LABELS
    )
    
    # ========================================
    # 3️⃣ ÉTIQUETTE DE STRATE COMBINÉE
    # ========================================
    # Créer un identifiant unique : "heure_classezone"
    # Exemple : "14_rare" = prise en charge à 14h dans une zone rare
    df['stratum'] = (
        df['pickup_hour'].astype(str) + "_" +
        df['zone_class'].astype(str)
    )
    
    # ========================================
    # 4️⃣ ÉCHANTILLONNAGE STRATIFIÉ PROPORTIONNEL
    # ========================================
    # Échantillonner SAMPLE_FRAC de chaque strate
    df_sample = (
        df
        .groupby('stratum', group_keys=False)
        .apply(lambda x: x.sample(
            frac=SAMPLE_FRAC,
            random_state=RANDOM_STATE
        ))
    )
    
    # ========================================
    # 5️⃣ NETTOYAGE
    # ========================================
    # Supprimer les colonnes temporaires de stratification
    df_sample.drop(
        columns=['pickup_hour', 'zone_class', 'stratum'],
        inplace=True
    )
    
    sample_parts.append(df_sample)
    
    # ========================================
    # 6️⃣ MISE À JOUR DE LA PROGRESSION
    # ========================================
    progress['completed'] = (i + 1) / n_files
    elapsed = time.time() - start_time
    progress['eta'] = elapsed / progress['completed'] * (1 - progress['completed'])

print(f"\nTraité {len(sample_parts)} fichiers")

[Échantillonnage stratifié hybride] 0.0% | CPU: 61.5% | RAM: 905.0 MB | ETA: infs
[Échantillonnage stratifié hybride] 0.0% | CPU: 66.7% | RAM: 1179.9 MB | ETA: infs


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_141376\4148316713.py:47: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(


[Échantillonnage stratifié hybride] 2.8% | CPU: 57.5% | RAM: 873.8 MB | ETA: 171.1s
[Échantillonnage stratifié hybride] 2.8% | CPU: 57.6% | RAM: 881.8 MB | ETA: 171.1s


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_141376\4148316713.py:47: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(


[Échantillonnage stratifié hybride] 5.6% | CPU: 57.9% | RAM: 776.7 MB | ETA: 181.2s
[Échantillonnage stratifié hybride] 5.6% | CPU: 56.8% | RAM: 1295.6 MB | ETA: 181.2s
[Échantillonnage stratifié hybride] 5.6% | CPU: 61.9% | RAM: 1072.2 MB | ETA: 181.2s


...

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_141376\4148316713.py:47: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(


[Échantillonnage stratifié hybride] 94.4% | CPU: 69.4% | RAM: 2369.1 MB | ETA: 12.6s
[Échantillonnage stratifié hybride] 94.4% | CPU: 51.1% | RAM: 1806.9 MB | ETA: 12.6s
[Échantillonnage stratifié hybride] 94.4% | CPU: 62.0% | RAM: 1916.4 MB | ETA: 12.6s


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_141376\4148316713.py:47: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(


[Échantillonnage stratifié hybride] 97.2% | CPU: 60.2% | RAM: 1936.9 MB | ETA: 6.3s
[Échantillonnage stratifié hybride] 97.2% | CPU: 55.5% | RAM: 1816.7 MB | ETA: 6.3s
[Échantillonnage stratifié hybride] 97.2% | CPU: 56.4% | RAM: 1884.3 MB | ETA: 6.3s

Traité 36 fichiers


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_141376\4148316713.py:47: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(


## 💾 Étape 5 : Combiner et Sauvegarder les Résultats

Maintenant nous :
1. **Concaténons** toutes les parties échantillonnées en un seul DataFrame
2. **Sauvegardons** au format CSV
3. **Arrêtons** le thread de surveillance
4. **Affichons les statistiques**

In [5]:
progress['operation'] = 'Sauvegarde CSV'

# Combiner tous les échantillons
df_final = pd.concat(sample_parts, ignore_index=True)

# Sauvegarder en CSV
df_final.to_csv(OUTPUT_FILE, index=False)

# Arrêter la surveillance
monitoring = False

# Calculer les statistiques finales
elapsed = time.time() - start_time

print("\n" + "="*50)
print("✅ Échantillonnage Stratifié Hybride Terminé")
print("="*50)
print(f"Taille finale de l'échantillon : {len(df_final):,} lignes")
print(f"Temps d'exécution total : {elapsed:.2f} secondes")
print(f"Sortie sauvegardée dans : {OUTPUT_FILE}")
print("="*50)

[Sauvegarde CSV] 100.0% | CPU: 51.2% | RAM: 2052.0 MB | ETA: 0.0s
[Sauvegarde CSV] 100.0% | CPU: 67.1% | RAM: 2069.5 MB | ETA: 0.0s
[Sauvegarde CSV] 100.0% | CPU: 67.1% | RAM: 2069.5 MB | ETA: 0.0s
[Sauvegarde CSV] 100.0% | CPU: 77.6% | RAM: 2069.4 MB | ETA: 0.0s
[Sauvegarde CSV] 100.0% | CPU: 70.4% | RAM: 2071.3 MB | ETA: 0.0s
[Sauvegarde CSV] 100.0% | CPU: 75.6% | RAM: 2069.4 MB | ETA: 0.0s
[Sauvegarde CSV] 100.0% | CPU: 73.4% | RAM: 2069.4 MB | ETA: 0.0s
[Sauvegarde CSV] 100.0% | CPU: 78.5% | RAM: 2069.4 MB | ETA: 0.0s
[Sauvegarde CSV] 100.0% | CPU: 77.5% | RAM: 2069.4 MB | ETA: 0.0s
[Sauvegarde CSV] 100.0% | CPU: 83.5% | RAM: 2069.4 MB | ETA: 0.0s
[Sauvegarde CSV] 100.0% | CPU: 82.7% | RAM: 2074.3 MB | ETA: 0.0s
[Sauvegarde CSV] 100.0% | CPU: 83.8% | RAM: 2069.4 MB | ETA: 0.0s
[Sauvegarde CSV] 100.0% | CPU: 94.6% | RAM: 2074.3 MB | ETA: 0.0s
[Sauvegarde CSV] 100.0% | CPU: 92.7% | RAM: 1928.6 MB | ETA: 0.0s

✅ Échantillonnage Stratifié Hybride Terminé
Taille finale de l'échantillon 

## 📊 Étape 6 : Vérifier la Qualité de l'Échantillon (Optionnel)

Vérifions la distribution de notre échantillon pour s'assurer que la stratification a fonctionné correctement.

In [6]:
# Afficher les statistiques de base
print("Aperçu de l'échantillon :")
print(df_final.info())

print("\nPremières lignes :")
print(df_final.head())

# Vérifier la distribution temporelle
if 'tpep_pickup_datetime' in df_final.columns:
    df_final['hour'] = pd.to_datetime(df_final['tpep_pickup_datetime']).dt.hour
    print("\nDistribution horaire :")
    print(df_final['hour'].value_counts().sort_index())
    
# Vérifier la distribution spatiale
if 'PULocationID' in df_final.columns:
    print("\nTop 10 des lieux de prise en charge :")
    print(df_final['PULocationID'].value_counts().head(10))

Aperçu de l'échantillon :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1191357 entries, 0 to 1191356
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   VendorID               1191357 non-null  int64         
 1   tpep_pickup_datetime   1191357 non-null  datetime64[us]
 2   tpep_dropoff_datetime  1191357 non-null  datetime64[us]
 3   passenger_count        1124628 non-null  float64       
 4   trip_distance          1191357 non-null  float64       
 5   RatecodeID             1124628 non-null  float64       
 6   store_and_fwd_flag     1124628 non-null  object        
 7   PULocationID           1191357 non-null  int64         
 8   DOLocationID           1191357 non-null  int64         
 9   payment_type           1191357 non-null  int64         
 10  fare_amount            1191357 non-null  float64       
 11  extra                  1191357 non-null  float64       
 12  mt

---


---

## 📚 Ressources Supplémentaires

- [Échantillonnage Stratifié Pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html)
- [Théorie de l'Échantillonnage Stratifié](https://fr.wikipedia.org/wiki/%C3%89chantillonnage_stratifi%C3%A9)
- [Documentation Dataset NYC Taxi](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)